In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
import string
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import scipy.sparse as sp
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from gensim.models import KeyedVectors
from gensim.summarization import summarize
from gensim.summarization import keywords
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from string import punctuation
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cancertreatment/stage2_test_text.csv
/kaggle/input/cancertreatment/test_text
/kaggle/input/cancertreatment/stage_2_private_solution.csv
/kaggle/input/cancertreatment/stage2_sample_submission.csv
/kaggle/input/cancertreatment/stage1_solution_filtered.csv
/kaggle/input/cancertreatment/test_variants
/kaggle/input/cancertreatment/stage2_test_variants.csv
/kaggle/input/cancer-diagnosis/training_text
/kaggle/input/cancer-diagnosis/training_variants


In [3]:
training_text = pd.read_csv("../input/cancer-diagnosis/training_text",sep="\|\|", header=None, skiprows=1, names=["ID","Text"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
training_text.head(5)

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [5]:
training_text.shape

(3321, 2)

In [6]:
training_variants = pd.read_csv("../input/cancer-diagnosis/training_variants")

In [7]:
training_variants.head(5)

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [8]:
#Merging variants and text on ID

In [9]:
training_merge = training_variants.merge(training_text,left_on="ID",right_on="ID")

In [10]:
training_merge.head(5)
#Now we have the data

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [11]:
training_merge.shape

(3321, 5)

In [12]:
test_text = pd.read_csv("../input/cancertreatment/stage2_test_text.csv",sep="\|\|", header=None, skiprows=1, names=["ID","Text"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [13]:
test_text.shape

(986, 2)

In [14]:
test_variants = pd.read_csv("../input/cancertreatment/stage2_test_variants.csv")

In [15]:
test_variants.shape

(986, 3)

In [16]:
test_merge = test_variants.merge(test_text,left_on="ID",right_on="ID")

In [17]:
# Let's understand the type of values present in each column of our dataframe 'train_merge' dataframe.
test_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986 entries, 0 to 985
Data columns (total 4 columns):
ID           986 non-null int64
Gene         986 non-null object
Variation    986 non-null object
Text         986 non-null object
dtypes: int64(1), object(3)
memory usage: 38.5+ KB


In [18]:
training_merge.describe(include='all')

,ID,Gene,Variation,Class,Text
count,3321.000000,3321,3321,3321.000000,3316
unique,NaN,264,2996,NaN,1920
top,NaN,BRCA1,Truncating Mutations,NaN,The PTEN (phosphatase and tensin homolog) phos...
freq,NaN,264,93,NaN,53
mean,1660.000000,NaN,NaN,4.365854,NaN
std,958.834449,NaN,NaN,2.309781,NaN
min,0.000000,NaN,NaN,1.000000,NaN
25%,830.000000,NaN,NaN,2.000000,NaN
50%,1660.000000,NaN,NaN,4.000000,NaN
75%,2490.000000,NaN,NaN,7.000000,NaN


In [19]:
training_merge.isnull().sum()

ID           0
Gene         0
Variation    0
Class        0
Text         5
dtype: int64

In [20]:
training_merge.columns

Index(['ID', 'Gene', 'Variation', 'Class', 'Text'], dtype='object')

In [21]:
training_merge["Text_num_words"] = training_merge["Text"].apply(lambda x: len(str(x).split()) )
training_merge["Text_num_chars"] = training_merge["Text"].apply(lambda x: len(str(x)) )

In [22]:
training_merge['Variation'].describe()

count                     3321
unique                    2996
top       Truncating Mutations
freq                        93
Name: Variation, dtype: object

In [23]:
training_merge['Gene'].describe()

count      3321
unique      264
top       BRCA1
freq        264
Name: Gene, dtype: object

In [24]:
#Tokenzing-splitting up a larger body of text into smaller lines, words or even creating words 

In [25]:
#imputing gene row value to null data of text rows as for all other columns, Gene values are present in Text data
training_merge['Text'] = training_merge.apply(lambda row: row['Gene'] if pd.isnull(row['Text']) else row['Text'],axis=1)

In [26]:
training_merge.isnull().sum()

ID                0
Gene              0
Variation         0
Class             0
Text              0
Text_num_words    0
Text_num_chars    0
dtype: int64

In [27]:
X = training_merge[['ID', 'Gene', 'Variation', 'Class', 'Text']]
y = training_merge['Class']

In [28]:
training_merge.drop(["Text_num_words","Text_num_chars"],axis=1,inplace=True)

In [29]:
stop_words = set(stopwords.words('english'))

In [30]:
def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
        training_merge[col][ind] = string

In [31]:
for index, row in training_merge.iterrows():
    if type(row['Text']) is str:
        data_text_preprocess(row['Text'], index, 'Text')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
training_merge.shape

(3321, 5)

In [33]:
training_merge.head()

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...
1,1,CBL,W802*,2,abstract background non small cell lung cancer...
2,2,CBL,Q249E,2,abstract background non small cell lung cancer...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b lineag...


In [34]:
training_merge['Text'][0]

'cyclin dependent kinases cdks regulate variety fundamental cellular processes cdk10 stands one last orphan cdks activating cyclin identified kinase activity revealed previous work shown cdk10 silencing increases ets2 v ets erythroblastosis virus e26 oncogene homolog 2 driven activation mapk pathway confers tamoxifen resistance breast cancer cells precise mechanisms cdk10 modulates ets2 activity generally functions cdk10 remain elusive demonstrate cdk10 cyclin dependent kinase identifying cyclin activating cyclin cyclin orphan cyclin product fam58a whose mutations cause star syndrome human developmental anomaly whose features include toe syndactyly telecanthus anogenital renal malformations show star syndrome associated cyclin mutants unable interact cdk10 cyclin silencing phenocopies cdk10 silencing increasing c raf conferring tamoxifen resistance breast cancer cells cdk10 cyclin phosphorylates ets2 vitro cells positively controls ets2 degradation proteasome ets2 protein levels increa

In [35]:
lda = LatentDirichletAllocation(n_components=9, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
tfA_vectorizer = CountVectorizer(stop_words='english')
tf_A = tfA_vectorizer.fit_transform(training_merge['Text'])

In [37]:
lda.fit(tf_A)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
tfA_feature_names = tfA_vectorizer.get_feature_names()
tfA_feature_names

['00',
 '000',
 '0000',
 '0000000000000188',
 '00000001',
 '0000001',
 '000000100',
 '00000096',
 '000001',
 '000002',
 '000003',
 '00000397',
 '000005',
 '000007',
 '000009',
 '00000o0',
 '00001',
 '000011',
 '0000112',
 '000012',
 '00001311',
 '0000149311',
 '000017',
 '0000208',
 '000022',
 '00002644',
 '00003',
 '00003450',
 '000038',
 '00004',
 '000042',
 '000044',
 '00005',
 '000050',
 '000051',
 '0000546',
 '000059',
 '00006',
 '0000614',
 '0000645',
 '00006559',
 '000077',
 '000077nm',
 '00008',
 '0000839',
 '00009',
 '000096',
 '0000965',
 '0000b',
 '0000o',
 '0001',
 '000100020003',
 '000102',
 '0001042',
 '000109nm',
 '00011',
 '000115',
 '0001171',
 '00012',
 '0001217',
 '000132',
 '000135',
 '00014',
 '000141',
 '000148',
 '00015',
 '00016',
 '000161',
 '000170',
 '000175',
 '000179',
 '00018',
 '000182',
 '000183',
 '00019',
 '000193',
 '000194',
 '0001a',
 '0001d',
 '0001e',
 '0001f',
 '0001g',
 '0001h',
 '0002',
 '000202464',
 '000205',
 '0002050',
 '00020503',
 '000207

In [39]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [40]:
print_top_words(lda, tfA_feature_names, 10)

Topic #0:
mutations patients mutation egfr cancer kit cell exon tumor cells
Topic #1:
cells mutations kinase raf cell activation activity figure mutants mutant
Topic #2:
cells cell ras et al fig expression protein figure cancer
Topic #3:
fusion ar gene ret expression cells fig genes fusions prostate
Topic #4:
mutations binding pten protein activity dna fig residues domain structure
Topic #5:
brca1 variants cancer deleterious brct variant vus data brca2 functional
Topic #6:
et al cells cell figure expression gene genes mice aml
Topic #7:
braf alk craf al fgfr et neuroblastoma fgfr3 mek mkk4
Topic #8:
tsc2 mtor tsc1 variants al et mutations amino type wild



In [41]:
print(tf_A)

  (0, 46120)	122
  (0, 49483)	6
  (0, 83765)	2
  (0, 38832)	7
  (0, 121433)	2
  (0, 147416)	3
  (0, 62772)	2
  (0, 39163)	3
  (0, 114735)	2
  (0, 38757)	129
  (0, 132915)	1
  (0, 104832)	3
  (0, 21654)	2
  (0, 76617)	8
  (0, 83734)	20
  (0, 21744)	11
  (0, 122852)	4
  (0, 114203)	5
  (0, 150293)	2
  (0, 129684)	14
  (0, 130231)	22
  (0, 78304)	2
  (0, 57594)	70
  (0, 57592)	2
  (0, 57271)	2
  :	:
  (3320, 31472)	5
  (3320, 31464)	2
  (3320, 31467)	2
  (3320, 31468)	2
  (3320, 31473)	2
  (3320, 136760)	1
  (3320, 97515)	1
  (3320, 151987)	1
  (3320, 147994)	1
  (3320, 131510)	1
  (3320, 20831)	1
  (3320, 149318)	1
  (3320, 100567)	1
  (3320, 31778)	1
  (3320, 29057)	1
  (3320, 5854)	1
  (3320, 138375)	1
  (3320, 40406)	1
  (3320, 84450)	1
  (3320, 85402)	1
  (3320, 125402)	1
  (3320, 63594)	2
  (3320, 141829)	1
  (3320, 21512)	1
  (3320, 34539)	1


In [42]:
tfB_vectorizer = CountVectorizer(stop_words='english')
tf_B = tfB_vectorizer.fit_transform(training_merge['Gene'])
lda.fit(tf_B)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [43]:
tfB_feature_names = tfB_vectorizer.get_feature_names()
tfB_feature_names

['abl1',
 'acvr1',
 'ago2',
 'akt1',
 'akt2',
 'akt3',
 'alk',
 'apc',
 'ar',
 'araf',
 'arid1a',
 'arid1b',
 'arid2',
 'arid5b',
 'asxl1',
 'asxl2',
 'atm',
 'atr',
 'atrx',
 'aurka',
 'aurkb',
 'axin1',
 'axl',
 'b2m',
 'bap1',
 'bard1',
 'bcl10',
 'bcl2',
 'bcl2l11',
 'bcor',
 'braf',
 'brca1',
 'brca2',
 'brd4',
 'brip1',
 'btk',
 'card11',
 'carm1',
 'casp8',
 'cbl',
 'ccnd1',
 'ccnd2',
 'ccnd3',
 'ccne1',
 'cdh1',
 'cdk12',
 'cdk4',
 'cdk6',
 'cdk8',
 'cdkn1a',
 'cdkn1b',
 'cdkn2a',
 'cdkn2b',
 'cdkn2c',
 'cebpa',
 'chek2',
 'cic',
 'crebbp',
 'ctcf',
 'ctla4',
 'ctnnb1',
 'ddr2',
 'dicer1',
 'dnmt3a',
 'dnmt3b',
 'dusp4',
 'egfr',
 'eif1ax',
 'elf3',
 'ep300',
 'epas1',
 'epcam',
 'erbb2',
 'erbb3',
 'erbb4',
 'ercc2',
 'ercc3',
 'ercc4',
 'erg',
 'errfi1',
 'esr1',
 'etv1',
 'etv6',
 'ewsr1',
 'ezh2',
 'fam58a',
 'fanca',
 'fancc',
 'fat1',
 'fbxw7',
 'fgf19',
 'fgf3',
 'fgf4',
 'fgfr1',
 'fgfr2',
 'fgfr3',
 'fgfr4',
 'flt1',
 'flt3',
 'foxa1',
 'foxl2',
 'foxo1',
 'foxp1',
 'f

In [44]:
print_top_words(lda, tfB_feature_names, 10)

Topic #0:
notch1 fgfr3 mlh1 nfe2l2 ctnnb1 pdgfrb spop fbxw7 ddr2 akt2
Topic #1:
braf tsc2 jak2 abl1 ar pms2 smad2 cdk12 foxa1 crebbp
Topic #2:
pten fgfr2 met smad4 tsc1 nras ppp2r1a ezh2 smarca4 epas1
Topic #3:
egfr kras cdkn2a vhl idh1 keap1 rb1 nf2 rac1 med12
Topic #4:
tp53 ntrk1 msh2 raf1 stk11 ptprd cic rasa1 bcor chek2
Topic #5:
brca2 erbb2 pdgfra mtor ros1 smad3 rhoa mpl erg cdkn1a
Topic #6:
ret pik3ca runx1 ptpn11 akt1 jak1 nf1 cbl ptprt tet2
Topic #7:
kit alk flt3 esr1 card11 sf3b1 kdr map2k4 pim1 pik3r2
Topic #8:
brca1 map2k1 smo pik3r1 hras erbb4 ercc2 myc fanca araf



In [45]:
tfC_vectorizer = CountVectorizer(stop_words='english')
tf_C = tfC_vectorizer.fit_transform(training_merge['Variation'])
lda.fit(tf_C)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [46]:
tfC_feature_names = tfC_vectorizer.get_feature_names()
tfC_feature_names

['11',
 '126',
 '13',
 '17',
 '19',
 '1_2009trunc',
 '20',
 '2010_2471trunc',
 '256_286trunc',
 '385_418del',
 '422_605trunc',
 '51',
 '533_534del',
 '534_536del',
 '550_592del',
 '560_561inser',
 '596_619splice',
 '6a',
 '963_d1010splice',
 '981_1028splice',
 'a1020v',
 'a1022e',
 'a1065t',
 'a1066v',
 'a1099t',
 'a111p',
 'a1131t',
 'a113_splice',
 'a1170v',
 'a11_g12insga',
 'a1200v',
 'a120s',
 'a121e',
 'a121p',
 'a121v',
 'a122',
 'a1234t',
 'a126d',
 'a126g',
 'a126s',
 'a126v',
 'a134d',
 'a1374v',
 'a1459p',
 'a146t',
 'a146v',
 'a148t',
 'a149p',
 'a1519t',
 'a151t',
 'a159t',
 'a161s',
 'a161t',
 'a1669s',
 'a1685s',
 'a1701p',
 'a1708e',
 'a1708v',
 'a171v',
 'a1752p',
 'a1752v',
 'a1789s',
 'a1789t',
 'a1823t',
 'a1830t',
 'a1843p',
 'a1843t',
 'a18d',
 'a197t',
 'a19v',
 'a2034v',
 'a205t',
 'a209t',
 'a211d',
 'a232v',
 'a2351g',
 'a23e',
 'a2425t',
 'a246p',
 'a263v',
 'a2643g',
 'a2717s',
 'a272v',
 'a2770t',
 'a290t',
 'a298t',
 'a339v',
 'a347t',
 'a349p',
 'a34d',
 

In [47]:
print_top_words(lda, tfC_feature_names, 10)

Topic #0:
q61h c618r y42c k117n q22k g13c h78q e768d g325a l790f
Topic #1:
g12v t58i g13v i31m r170w e542k p173s y113 v648i variant
Topic #2:
e17k q209l exon 20 a146v t73i r173c g67r a883t v804m
Topic #3:
fusion pdgfrb ntrk1 braf ewsr1 ros1 jak2 fgfr2 fgfr1 alk
Topic #4:
amplification overexpression f28l s308a y806c r715g c634s r80c r661w s904f
Topic #5:
deletion exon q61k y64a g13d 19 f384l t286a r841k r166q
Topic #6:
mutations truncating exon domain e330k a883f c706f r833c s891a e921k
Topic #7:
fusions g12d m1r g12s p34r v710a c609y t654m g42r l63v
Topic #8:
q61r q61l g12c a146t g12a s222d t167a v321m p130s deletions



In [48]:
X_final = sp.hstack((tf_A,tf_B,tf_C))

In [49]:
X_final.shape

(3321, 156140)

In [50]:
X_final

<3321x156140 sparse matrix of type '<class 'numpy.int64'>'
	with 5141706 stored elements in COOrdinate format>

In [51]:
print(X_final)

  (0, 46120)	122
  (0, 49483)	6
  (0, 83765)	2
  (0, 38832)	7
  (0, 121433)	2
  (0, 147416)	3
  (0, 62772)	2
  (0, 39163)	3
  (0, 114735)	2
  (0, 38757)	129
  (0, 132915)	1
  (0, 104832)	3
  (0, 21654)	2
  (0, 76617)	8
  (0, 83734)	20
  (0, 21744)	11
  (0, 122852)	4
  (0, 114203)	5
  (0, 150293)	2
  (0, 129684)	14
  (0, 130231)	22
  (0, 78304)	2
  (0, 57594)	70
  (0, 57592)	2
  (0, 57271)	2
  :	:
  (3303, 154882)	1
  (3304, 155198)	1
  (3305, 155602)	1
  (3306, 155974)	1
  (3307, 155772)	1
  (3307, 154745)	1
  (3308, 155173)	1
  (3309, 153440)	1
  (3310, 153277)	1
  (3311, 153920)	1
  (3311, 155430)	1
  (3311, 153824)	1
  (3312, 153920)	1
  (3312, 155759)	1
  (3312, 155430)	1
  (3313, 154207)	1
  (3314, 154048)	1
  (3315, 153920)	1
  (3315, 155430)	1
  (3315, 155431)	1
  (3316, 153441)	1
  (3317, 153157)	1
  (3318, 153921)	1
  (3319, 155387)	1
  (3320, 154448)	1


In [52]:
Y_final=training_merge['Class']

In [53]:
X_train, X_test, Y_train, Y_test = train_test_split(X_final,Y_final,random_state=1)

In [54]:
clfA = MultinomialNB().fit(X_train, Y_train)
predicted= clfA.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(Y_test, predicted))
print(classification_report(Y_test,predicted))

MultinomialNB Accuracy: 0.5824308062575211
              precision    recall  f1-score   support

           1       0.50      0.61      0.55       142
           2       0.41      0.46      0.43       117
           3       0.61      0.41      0.49        27
           4       0.77      0.52      0.62       170
           5       0.33      0.48      0.39        50
           6       0.71      0.57      0.64        61
           7       0.68      0.70      0.69       250
           8       1.00      0.20      0.33         5
           9       0.73      0.89      0.80         9

    accuracy                           0.58       831
   macro avg       0.64      0.54      0.55       831
weighted avg       0.61      0.58      0.59       831



In [55]:
# defining parameter range 
param_grid = {'C': [0.1, 1], #10, 100, 1000],  
              'gamma': [1, 0.1, 0.01], #0.001, 0.0001], 
              'kernel': ['linear']}  
  
gridA = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,) 
  
# fitting the model for grid search 
gridA.fit(X_train, Y_train) 

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.574, total= 4.6min
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.6min remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.564, total= 4.3min
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.9min remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.570, total= 4.5min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.574, total= 4.6min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.564, total= 4.3min
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.570, total= 4.5min
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.574, total= 4.5min
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.564, total= 4.3min
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.570, total= 4.6min
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 83.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [56]:
gridA_predictions = gridA.predict(X_test) 
  
# print classification report 
print(classification_report(Y_test, gridA_predictions)) 

              precision    recall  f1-score   support

           1       0.55      0.54      0.55       142
           2       0.54      0.55      0.54       117
           3       0.33      0.30      0.31        27
           4       0.66      0.68      0.67       170
           5       0.40      0.42      0.41        50
           6       0.68      0.74      0.71        61
           7       0.74      0.72      0.73       250
           8       0.33      0.40      0.36         5
           9       0.75      0.67      0.71         9

    accuracy                           0.62       831
   macro avg       0.55      0.56      0.55       831
weighted avg       0.62      0.62      0.62       831



In [57]:
#Using weighted TD-IDF

In [58]:
vectorizer_NA = TfidfVectorizer(stop_words='english',ngram_range=(2,2))
tf_NA = vectorizer_NA.fit_transform(training_merge['Text'])

In [59]:
lda.fit(tf_NA)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
tf_NA_feature_names = vectorizer_NA.get_feature_names()
tf_NA_feature_names

['00 00',
 '00 000',
 '00 000000100',
 '00 0001',
 '00 000183',
 '00 0003',
 '00 0004',
 '00 0005',
 '00 000532',
 '00 000o0',
 '00 001',
 '00 0010',
 '00 001565',
 '00 0018',
 '00 002',
 '00 0021',
 '00 0026',
 '00 003',
 '00 004',
 '00 004507validation',
 '00 004678',
 '00 0055',
 '00 006',
 '00 009',
 '00 01',
 '00 010',
 '00 011',
 '00 012',
 '00 015',
 '00 01we',
 '00 02',
 '00 02181',
 '00 025',
 '00 027',
 '00 029',
 '00 03',
 '00 031',
 '00 032',
 '00 034',
 '00 04',
 '00 048',
 '00 05',
 '00 050',
 '00 06',
 '00 0600',
 '00 07',
 '00 072',
 '00 08',
 '00 082',
 '00 083',
 '00 10',
 '00 1010',
 '00 11',
 '00 111',
 '00 112',
 '00 116',
 '00 12',
 '00 120',
 '00 128',
 '00 13',
 '00 1399',
 '00 14',
 '00 143',
 '00 15',
 '00 16',
 '00 17',
 '00 171',
 '00 181',
 '00 19',
 '00 198',
 '00 20',
 '00 2004',
 '00 21',
 '00 227',
 '00 229',
 '00 23',
 '00 24',
 '00 248',
 '00 252',
 '00 256',
 '00 26',
 '00 268',
 '00 27',
 '00 29',
 '00 298',
 '00 30',
 '00 31',
 '00 32',
 '00 326',


In [61]:
print_top_words(lda, tf_NA_feature_names, 10)

Topic #0:
egfp p53 tetramer formation dsred positive egfp signal fraction dsred dsred signal oligomeric status tetrameric structure p53td peptides cellular extract
Topic #1:
human milk product ion uhplc qqtof qqtof ms fish meat ms ms confirms identity sediment fish achieved ultrahigh possibilities scan
Topic #2:
egfr mutations et al egfr tki egfr mutation egfr 768 cbl e2 lung cancer cell lines nsclc patients rnase iiib
Topic #3:
et al wild type cell lines author manuscript brca1 brct supplementary fig pten mutations brca1 brca2 vus neutral family history
Topic #4:
exon 20 egfr mutations 20 insertion egfr mutation cbl mutations cbl e2 egfr tkis wild type prostate cancer 20 insertions
Topic #5:
flag elf3 mt ii ii 108 108 56 f9 differentiated 235 260 sodium metabisulfite elf3 235 elf3 n270 pancreatic necrosis
Topic #6:
p53 fragment dna site 71 330 acids 71 dna sites quarter sites p5371 330 pentamer quarter pair position c277q c277r
Topic #7:
rnase iii rnase iiib hs dicer egfr mutations ex

In [62]:
vectorizer_NB = TfidfVectorizer(stop_words='english',ngram_range=(2,2))
tf_NB = vectorizer_NB.fit_transform(training_merge['Variation'])

In [63]:
lda.fit(tf_NB)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [64]:
tf_NB_feature_names = vectorizer_NB.get_feature_names()
tf_NB_feature_names

['11 mutations',
 '13 deletion',
 '19 deletion',
 '19 insertion',
 '20 insertion',
 '20 insertions',
 'abl1 fusion',
 'acpp pik3cb',
 'agk braf',
 'ahcyl1 fusion',
 'akap9 braf',
 'akt3 fusion',
 'alk fusion',
 'ar v7',
 'atf1 fusion',
 'atf7ip jak2',
 'atf7ip pdgfrb',
 'atg7 raf1',
 'baiap2l1 fusion',
 'bcan ntrk1',
 'bcl2 fusion',
 'bcor ccnb3',
 'bcor fusion',
 'bcor rara',
 'bcr abl1',
 'bcr fgfr1',
 'bcr jak2',
 'bcr pdgfra',
 'bicc1 fusion',
 'bin2 pdgfrb',
 'binding domain',
 'braf fusion',
 'brd4 nut',
 'btbd1 ntrk3',
 'cad alk',
 'ccdc170 fusion',
 'ccdc6 fusion',
 'ccdc6 ros1',
 'ccnb3 fusion',
 'ccnd1 igh',
 'cd74 fusion',
 'cd74 ntrk1',
 'cd74 ros1',
 'cep110 fgfr1',
 'cep85l pdgfrb',
 'cep85l ros1',
 'chtop ntrk1',
 'cic dux4',
 'copy number',
 'cpeb1 ntrk3',
 'creb1 fusion',
 'cul1 braf',
 'cux1 fgfr1',
 'ddit3 fusion',
 'deletion insertion',
 'delta ntrk1',
 'dna binding',
 'domain deletions',
 'domain insertions',
 'domain missense',
 'dux4 fusion',
 'ebf1 pdgfrb',
 'eg

In [65]:
print_top_words(lda, tf_NB_feature_names, 10)

Topic #0:
promoter hypermethylation ntrk1 fusion braf fusion mprip ntrk1 tfg ntrk1 evi1 fusion runx1 evi1 pdgfrb fusion kiaa1549 braf mkrn1 braf
Topic #1:
braf fusion notch1 fusion ccdc6 fusion q2405rfs 17 myc nick ret ccdc6 sec16a1 notch1 mir143 notch1 akap9 braf snd1 braf
Topic #2:
pdgfrb fusion ros1 fusion fgfr1 fusion lmna ntrk1 ar v7 tel runx1 runx1 fusion akt3 fusion magi3 akt3 exon 11
Topic #3:
exon mutations ntrk1 fusion fgfr1 fusion delta ntrk1 tpr ntrk1 r177pfs 126 trkaiii splice splice variant ros1 fusion bcor fusion
Topic #4:
etv1 fusion braf fusion pdgfrb fusion exon 20 ewsr1 etv1 tmprss2 etv1 20 insertions kif5b ret ret fusion esrp1 raf1
Topic #5:
truncating mutations etv4 fusion ros1 fusion mutations pest pest domain rara fusion bcor rara mutations upstream upstream transactivation transactivation domain
Topic #6:
alk fusion jak2 fusion p291qfs 51 cd74 ntrk1 nsd1 nup98 nup98 fusion atg7 raf1 ntrk1 fusion raf1 fusion pcm1 jak2
Topic #7:
pdgfrb fusion ntrk3 fusion ntrk1 fu

In [66]:
XB_final = sp.hstack((tf_NA,tf_B,tf_NB))

In [67]:
XB_train, XB_test, Y_train, Y_test = train_test_split(XB_final,Y_final,random_state=1)

In [68]:
clfB = MultinomialNB().fit(XB_train, Y_train)
predicted= clfB.predict(XB_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(Y_test, predicted))
print(classification_report(Y_test,predicted))

MultinomialNB Accuracy: 0.6089049338146811
              precision    recall  f1-score   support

           1       0.61      0.56      0.58       142
           2       0.93      0.12      0.21       117
           3       0.00      0.00      0.00        27
           4       0.65      0.71      0.68       170
           5       0.40      0.24      0.30        50
           6       0.76      0.52      0.62        61
           7       0.58      0.98      0.73       250
           8       0.00      0.00      0.00         5
           9       1.00      0.22      0.36         9

    accuracy                           0.61       831
   macro avg       0.55      0.37      0.39       831
weighted avg       0.63      0.61      0.55       831



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [69]:
t_id = 0
text = training_merge.loc[t_id, 'Text']

word_scores = keywords(text, words=5, scores=True, split=True, lemmatize=True)
summary = summarize(text, word_count=100)

print('ID [{}]\nKeywords: [{}]\nSummary: [{}]'.format(t_id, word_scores, summary))

ValueError: input must have more than one sentence

In [70]:
custom_words = ["fig", "figure", "et", "al", "al.", "also",
                "data", "analyze", "study", "table", "using",
                "method", "result", "conclusion", "author", 
                "find", "found", "show", '"', "’", "“", "”"]

stop_words = set(stopwords.words('english') + list(punctuation) + custom_words)
wordnet_lemmatizer = WordNetLemmatizer()

class_corpus = training_merge.groupby('Class').apply(lambda x: x['Text'].str.cat())
class_corpus = class_corpus.apply(lambda x: Counter(
    [wordnet_lemmatizer.lemmatize(w) 
     for w in word_tokenize(x) 
     if w.lower() not in stop_words and not w.isdigit()]
))

In [71]:
training_merge.shape

(3321, 5)

In [72]:
X_one=training_merge[training_merge['Class']==1]
X_two=training_merge[training_merge['Class']==2]
X_thr=training_merge[training_merge['Class']==3]
X_for=training_merge[training_merge['Class']==4]
X_fiv=training_merge[training_merge['Class']==5]
X_six=training_merge[training_merge['Class']==6]
X_sev=training_merge[training_merge['Class']==7]
X_eig=training_merge[training_merge['Class']==8]
X_nin=training_merge[training_merge['Class']==9]

In [73]:
X_one.head(3)

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...
7,7,CBL,Deletion,1,cbl negative regulator activated receptor tyro...
16,16,CBL,Truncating Mutations,1,determine residual cylindrical refractive erro...


In [74]:
X_one['Gene'].describe()

count      568
unique     142
top       TP53
freq        83
Name: Gene, dtype: object

In [75]:
X_oneg=X_one['Gene'].value_counts()

In [76]:
common_ga = set.intersection(set(X_one.Gene), set(X_two.Gene))
common_va = set.intersection(set(X_one.Variation), set(X_two.Variation))
common_gb = set.intersection(set(X_one.Gene), set(X_thr.Gene))
common_vb = set.intersection(set(X_one.Variation), set(X_thr.Variation))
common_gc = set.intersection(set(X_one.Gene), set(X_for.Gene))
common_vc = set.intersection(set(X_one.Variation), set(X_for.Variation))
common_gd = set.intersection(set(X_one.Gene), set(X_fiv.Gene))
common_vd = set.intersection(set(X_one.Variation), set(X_fiv.Variation))
common_ge = set.intersection(set(X_one.Gene), set(X_six.Gene))
common_ve = set.intersection(set(X_one.Variation), set(X_six.Variation))
common_gf = set.intersection(set(X_one.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_one.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_one.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_one.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_one.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_one.Variation), set(X_nin.Variation))

In [77]:
print(len(common_ga))
print(len(common_va))
print(len(common_gb))
print(len(common_vb))
print(len(common_gc))
print(len(common_vc))
print(len(common_gd))
print(len(common_vd))
print(len(common_ge))
print(len(common_ve))
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

32
2
17
0
67
4
38
0
30
2
26
1
5
1
4
0


In [78]:
common_gb = set.intersection(set(X_two.Gene), set(X_thr.Gene))
common_vb = set.intersection(set(X_two.Variation), set(X_thr.Variation))
common_gc = set.intersection(set(X_two.Gene), set(X_for.Gene))
common_vc = set.intersection(set(X_two.Variation), set(X_for.Variation))
common_gd = set.intersection(set(X_two.Gene), set(X_fiv.Gene))
common_vd = set.intersection(set(X_two.Variation), set(X_fiv.Variation))
common_ge = set.intersection(set(X_two.Gene), set(X_six.Gene))
common_ve = set.intersection(set(X_two.Variation), set(X_six.Variation))
common_gf = set.intersection(set(X_two.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_two.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_two.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_two.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_two.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_two.Variation), set(X_nin.Variation))

In [79]:
print(len(common_gb))
print(len(common_vb))
print(len(common_gc))
print(len(common_vc))
print(len(common_gd))
print(len(common_vd))
print(len(common_ge))
print(len(common_ve))
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

15
0
24
1
24
1
28
2
69
14
6
1
3
0


In [80]:
common_gc = set.intersection(set(X_thr.Gene), set(X_for.Gene))
common_vc = set.intersection(set(X_thr.Variation), set(X_for.Variation))
common_gd = set.intersection(set(X_thr.Gene), set(X_fiv.Gene))
common_vd = set.intersection(set(X_thr.Variation), set(X_fiv.Variation))
common_ge = set.intersection(set(X_thr.Gene), set(X_six.Gene))
common_ve = set.intersection(set(X_thr.Variation), set(X_six.Variation))
common_gf = set.intersection(set(X_thr.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_thr.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_thr.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_thr.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_thr.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_thr.Variation), set(X_nin.Variation))

In [81]:
print(len(common_gc))
print(len(common_vc))
print(len(common_gd))
print(len(common_vd))
print(len(common_ge))
print(len(common_ve))
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

15
1
14
1
9
0
15
0
2
0
1
0


In [82]:
common_gd = set.intersection(set(X_for.Gene), set(X_fiv.Gene))
common_vd = set.intersection(set(X_for.Variation), set(X_fiv.Variation))
common_ge = set.intersection(set(X_for.Gene), set(X_six.Gene))
common_ve = set.intersection(set(X_for.Variation), set(X_six.Variation))
common_gf = set.intersection(set(X_for.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_for.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_for.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_for.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_for.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_for.Variation), set(X_nin.Variation))

In [83]:
print(len(common_gd))
print(len(common_vd))
print(len(common_ge))
print(len(common_ve))
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

28
2
23
2
25
2
5
1
2
0


In [84]:
common_ge = set.intersection(set(X_fiv.Gene), set(X_six.Gene))
common_ve = set.intersection(set(X_fiv.Variation), set(X_six.Variation))
common_gf = set.intersection(set(X_fiv.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_fiv.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_fiv.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_fiv.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_fiv.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_fiv.Variation), set(X_nin.Variation))

In [85]:
print(len(common_ge))
print(len(common_ve))
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

26
1
21
1
4
0
2
0


In [86]:
common_gf = set.intersection(set(X_six.Gene), set(X_sev.Gene))
common_vf = set.intersection(set(X_six.Variation), set(X_sev.Variation))
common_gg = set.intersection(set(X_six.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_six.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_six.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_six.Variation), set(X_nin.Variation))

In [87]:
print(len(common_gf))
print(len(common_vf))
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

26
2
3
0
1
0


In [88]:
common_gg = set.intersection(set(X_sev.Gene), set(X_eig.Gene))
common_vg = set.intersection(set(X_sev.Variation), set(X_eig.Variation))
common_gh = set.intersection(set(X_sev.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_sev.Variation), set(X_nin.Variation))

In [89]:
print(len(common_gg))
print(len(common_vg))
print(len(common_gh))
print(len(common_vh))

6
0
4
0


In [90]:
common_gh = set.intersection(set(X_eig.Gene), set(X_nin.Gene))
common_vh = set.intersection(set(X_eig.Variation), set(X_nin.Variation))

In [91]:
print(len(common_gh))
print(len(common_vh))

3
0


One and four and two and five have way too many common genes and variations in common so we would club them together and keep the rest as it is

In [92]:
A=[1,4]
B=[2,5]
C=[3,6,7,9]

In [93]:
train_A=training_merge[training_merge['Class'].isin(A)]
train_B=training_merge[training_merge['Class'].isin(B)]
train_C=training_merge[training_merge['Class'].isin(C)]

In [94]:
Y_A=train_A['Class']
Y_B=train_B['Class']
Y_C=train_C['Class']

In [95]:
lda = LatentDirichletAllocation(n_components=9, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [96]:
tfAA_vectorizer = CountVectorizer(stop_words='english')
tf_AA = tfAA_vectorizer.fit_transform(train_A['Text'])
tfAB_vectorizer = CountVectorizer(stop_words='english')
tf_AB = tfAB_vectorizer.fit_transform(train_B['Text'])
tfAC_vectorizer = CountVectorizer(stop_words='english')
tf_AC = tfAC_vectorizer.fit_transform(train_C['Text'])

In [97]:
lda.fit(tf_AA)
lda.fit(tf_AB)
lda.fit(tf_AC)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [98]:
tfAA_feature_names = tfAA_vectorizer.get_feature_names()
tfAA_feature_names

['00',
 '000',
 '0000001',
 '000001',
 '000002',
 '000003',
 '00000397',
 '000005',
 '000007',
 '000009',
 '00001',
 '0000149311',
 '000017',
 '00003',
 '00003450',
 '000038',
 '00005',
 '000050',
 '000051',
 '0000546',
 '000059',
 '00006',
 '000077',
 '000077nm',
 '00008',
 '00009',
 '000096',
 '0000965',
 '0001',
 '000100020003',
 '000109nm',
 '00011',
 '00012',
 '0001217',
 '000132',
 '000135',
 '00014',
 '000141',
 '00016',
 '000161',
 '000170',
 '000179',
 '000183',
 '00019',
 '0001a',
 '0002',
 '000202464',
 '00021',
 '000240',
 '000242',
 '000249',
 '000249893',
 '00025',
 '000251',
 '000258',
 '000267',
 '000268',
 '00028',
 '00029',
 '0003',
 '0003052',
 '000321',
 '00034',
 '000351',
 '000368',
 '000369266',
 '00037',
 '000386',
 '000389',
 '0004',
 '000400403',
 '000418',
 '00042',
 '000420777',
 '000455',
 '000457',
 '00046',
 '00047',
 '000480',
 '000483',
 '000489',
 '0005',
 '00051',
 '0005137',
 '000526',
 '00053',
 '000532',
 '000535',
 '000539',
 '000545',
 '000546',


In [99]:
tfAB_feature_names = tfAB_vectorizer.get_feature_names()
tfAB_feature_names

['00',
 '000',
 '0000',
 '000001',
 '000002',
 '000007',
 '00001',
 '000011',
 '0000112',
 '000012',
 '0000208',
 '00002644',
 '00004',
 '000042',
 '000059',
 '0000614',
 '0000645',
 '00006559',
 '0000839',
 '00009',
 '0001',
 '0001042',
 '0001171',
 '00012',
 '000132',
 '00014',
 '000141',
 '000148',
 '00016',
 '000179',
 '00018',
 '000182',
 '00019',
 '000193',
 '0002',
 '0002050',
 '0002079',
 '0002158',
 '000245',
 '000249',
 '00025',
 '000251',
 '000267',
 '0003',
 '0003073',
 '000314',
 '0003254',
 '0003529',
 '000369266',
 '0004',
 '000435',
 '0004376',
 '0004798',
 '0004920',
 '000498',
 '0005',
 '0005044',
 '000508',
 '0005159',
 '0005214',
 '0005266',
 '0005309',
 '000535',
 '0005355',
 '00058',
 '0006',
 '0007',
 '000767',
 '0008',
 '000834',
 '00088',
 '00089',
 '0009',
 '000g',
 '000x',
 '000xg',
 '001',
 '0010',
 '001013415',
 '00103',
 '00106',
 '001070679',
 '001093',
 '001094320',
 '0011',
 '001105',
 '001106174',
 '001130145',
 '001134673',
 '001166213',
 '001172713',

In [100]:
tfAC_feature_names = tfAC_vectorizer.get_feature_names()
tfAC_feature_names

['00',
 '000',
 '0000',
 '0000000000000188',
 '00000001',
 '0000001',
 '000000100',
 '00000096',
 '000001',
 '000002',
 '000005',
 '000007',
 '000009',
 '00000o0',
 '00001',
 '000011',
 '0000112',
 '000012',
 '00001311',
 '0000208',
 '000022',
 '00002644',
 '00003',
 '000044',
 '00005',
 '00006',
 '0000614',
 '0000645',
 '00006559',
 '000077',
 '00008',
 '0000839',
 '0000b',
 '0000o',
 '0001',
 '000102',
 '0001042',
 '000115',
 '0001171',
 '000132',
 '000141',
 '000148',
 '00015',
 '00016',
 '000175',
 '000182',
 '00019',
 '000194',
 '0001d',
 '0001e',
 '0001f',
 '0001g',
 '0001h',
 '0002',
 '000205',
 '0002050',
 '00020503',
 '0002079',
 '00021',
 '00021427',
 '0002158',
 '000219',
 '000222',
 '00023490',
 '000236',
 '000245',
 '000249',
 '00025',
 '000261',
 '000264',
 '00027',
 '000278',
 '00028',
 '00029',
 '0003',
 '00030',
 '0003019',
 '0003073',
 '000314',
 '0003254',
 '00033',
 '00033740',
 '00034',
 '0003529',
 '000357129',
 '000369266',
 '00037',
 '0004',
 '000404',
 '0004376

In [101]:
print_top_words(lda, tfAC_feature_names, 10)

Topic #0:
egfr patients mutations exon mutation gefitinib response kit treatment 20
Topic #1:
ras al et cells cell mutations figure expression fig cyclin
Topic #2:
cells mutations cell kinase fig mutation figure activity activation mutant
Topic #3:
egfr brd4 mutations mastocytosis nd patients mutation u87mg egfrviii xxl
Topic #4:
jak2 e2 shp2 bard1 interaction ns i26a ptpn11 jh2 syndrome
Topic #5:
nrf2 keap1 jak2 fedratinib ruxolitinib card11 binding trk site substrate
Topic #6:
mtor mutations figure cells erbb3 mtorc1 cancer al mutants et
Topic #7:
brca1 variants cancer deleterious mutations data ar mutation variant analysis
Topic #8:
ret braf mutations patients palb2 map2k1 lch mtc pten chek2



In [102]:
tfBA_vectorizer = CountVectorizer(stop_words='english')
tf_BA = tfBA_vectorizer.fit_transform(train_A['Gene'])
tfBB_vectorizer = CountVectorizer(stop_words='english')
tf_BB = tfBB_vectorizer.fit_transform(train_B['Gene'])
tfBC_vectorizer = CountVectorizer(stop_words='english')
tf_BC = tfBC_vectorizer.fit_transform(train_C['Gene'])

In [103]:
lda.fit(tf_BA)
lda.fit(tf_BB)
lda.fit(tf_BC)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [104]:
tfBA_feature_names = tfBA_vectorizer.get_feature_names()
tfBA_feature_names

['ago2',
 'apc',
 'ar',
 'arid1a',
 'arid1b',
 'arid2',
 'arid5b',
 'asxl1',
 'asxl2',
 'atm',
 'atr',
 'atrx',
 'aurka',
 'axin1',
 'b2m',
 'bap1',
 'bard1',
 'bcl10',
 'bcl2l11',
 'bcor',
 'braf',
 'brca1',
 'brca2',
 'brip1',
 'btk',
 'carm1',
 'casp8',
 'cbl',
 'cdh1',
 'cdk12',
 'cdk6',
 'cdkn1a',
 'cdkn1b',
 'cdkn2a',
 'cdkn2b',
 'cdkn2c',
 'cebpa',
 'chek2',
 'cic',
 'crebbp',
 'ctcf',
 'ctla4',
 'dicer1',
 'dnmt3a',
 'egfr',
 'elf3',
 'ep300',
 'epcam',
 'erbb2',
 'ercc2',
 'ercc3',
 'ercc4',
 'errfi1',
 'etv6',
 'fam58a',
 'fanca',
 'fancc',
 'fat1',
 'fbxw7',
 'fgfr1',
 'fgfr2',
 'fgfr3',
 'flt3',
 'foxa1',
 'foxl2',
 'foxo1',
 'foxp1',
 'fubp1',
 'gata3',
 'gli1',
 'hist1h1c',
 'hla',
 'hnf1a',
 'idh1',
 'igf1r',
 'ikzf1',
 'inpp4b',
 'jak1',
 'kdm5c',
 'kdm6a',
 'keap1',
 'kit',
 'kmt2a',
 'kmt2c',
 'kmt2d',
 'knstrn',
 'lats1',
 'lats2',
 'map2k1',
 'map2k4',
 'map3k1',
 'med12',
 'mef2b',
 'men1',
 'mga',
 'mlh1',
 'msh2',
 'msh6',
 'ncor1',
 'nf1',
 'nf2',
 'notch1',
 'n

In [105]:
tfBB_feature_names = tfBB_vectorizer.get_feature_names()
tfBB_feature_names

['abl1',
 'acvr1',
 'ago2',
 'akt1',
 'akt2',
 'akt3',
 'alk',
 'ar',
 'araf',
 'atm',
 'aurkb',
 'axl',
 'braf',
 'brca1',
 'brca2',
 'btk',
 'card11',
 'cbl',
 'ccnd1',
 'ccnd2',
 'ccnd3',
 'cdk12',
 'crebbp',
 'ctnnb1',
 'ddr2',
 'dnmt3a',
 'dnmt3b',
 'egfr',
 'elf3',
 'ep300',
 'epas1',
 'erbb2',
 'erbb3',
 'erbb4',
 'ercc2',
 'erg',
 'esr1',
 'etv1',
 'etv6',
 'ewsr1',
 'fbxw7',
 'fgf19',
 'fgfr1',
 'fgfr2',
 'fgfr3',
 'flt3',
 'foxa1',
 'gata3',
 'gnas',
 'hras',
 'idh1',
 'igf1r',
 'ikbke',
 'jak2',
 'kdm5a',
 'kdm5c',
 'kdr',
 'kit',
 'klf4',
 'kmt2a',
 'kmt2c',
 'kras',
 'map2k1',
 'map2k2',
 'map2k4',
 'map3k1',
 'med12',
 'mef2b',
 'met',
 'mlh1',
 'mpl',
 'msh2',
 'msh6',
 'mtor',
 'myc',
 'mycn',
 'myod1',
 'nf2',
 'nfe2l2',
 'nfkbia',
 'nkx2',
 'notch1',
 'notch2',
 'nras',
 'nsd1',
 'ntrk1',
 'ntrk2',
 'ntrk3',
 'pdgfra',
 'pdgfrb',
 'pik3ca',
 'pik3cb',
 'pik3r2',
 'pim1',
 'pms2',
 'ptch1',
 'pten',
 'ptpn11',
 'raf1',
 'rb1',
 'ret',
 'rhoa',
 'rit1',
 'ros1',
 'smad2

In [106]:
tfBC_feature_names = tfBC_vectorizer.get_feature_names()
tfBC_feature_names

['abl1',
 'acvr1',
 'akt1',
 'akt2',
 'akt3',
 'alk',
 'ar',
 'araf',
 'aurka',
 'bcl2',
 'braf',
 'brca1',
 'brca2',
 'brd4',
 'brip1',
 'btk',
 'card11',
 'casp8',
 'cbl',
 'ccnd1',
 'ccnd3',
 'ccne1',
 'cdk4',
 'cdk6',
 'cdk8',
 'cdkn1a',
 'cdkn1b',
 'cdkn2a',
 'chek2',
 'cic',
 'ctcf',
 'ctnnb1',
 'ddr2',
 'dusp4',
 'egfr',
 'eif1ax',
 'ep300',
 'erbb2',
 'erbb3',
 'erbb4',
 'erg',
 'esr1',
 'etv1',
 'etv6',
 'ewsr1',
 'ezh2',
 'fbxw7',
 'fgf3',
 'fgf4',
 'fgfr1',
 'fgfr2',
 'fgfr3',
 'fgfr4',
 'flt1',
 'flt3',
 'foxa1',
 'foxl2',
 'gna11',
 'gnaq',
 'gnas',
 'hnf1a',
 'hras',
 'idh1',
 'idh2',
 'il7r',
 'jak1',
 'jak2',
 'jun',
 'kdr',
 'kit',
 'kmt2a',
 'kmt2b',
 'kras',
 'map2k1',
 'map2k2',
 'map3k1',
 'mapk1',
 'mdm2',
 'mdm4',
 'med12',
 'mef2b',
 'met',
 'mlh1',
 'mpl',
 'msh2',
 'mtor',
 'myc',
 'mycn',
 'myd88',
 'nf2',
 'nfe2l2',
 'nfkbia',
 'nkx2',
 'notch1',
 'nras',
 'ntrk1',
 'ntrk3',
 'nup93',
 'pak1',
 'pax8',
 'pdgfra',
 'pdgfrb',
 'pik3ca',
 'pik3cb',
 'pik3cd',
 

In [107]:
print_top_words(lda, tfBA_feature_names, 10)

Topic #0:
atr ar pik3r2 foxo1 cic setd2 fgfr3 braf asxl2 ago2
Topic #1:
atrx kmt2a bard1 arid1b sdhb lats1 nup93 rasa1 map2k1 ptch1
Topic #2:
cdkn2b fam58a knstrn men1 fgfr1 runx1 ptpn11 smad4 epcam kdm6a
Topic #3:
gli1 hnf1a ercc4 rb1 bcor ercc2 asxl1 shoc2 rnf43 foxa1
Topic #4:
nf2 ppp6c ercc3 mlh1 pim1 ptprd rad21 fgfr2 fubp1 idh1
Topic #5:
chek2 foxp1 idh1 cdkn1a map3k1 aurka smarcb1 keap1 erbb2 btk
Topic #6:
notch2 kit pms2 cdkn1b ctla4 arid1a elf3 notch1 ppp2r1a ctcf
Topic #7:
aurka crebbp arid5b igf1r pik3r1 msh2 rbm10 flt3 ntrk2 rad51d
Topic #8:
arid2 mga rad51c gata3 egfr bcl10 pik3r3 npm1 ntrk1 cdkn2a



In [108]:
print_top_words(lda, tfBC_feature_names, 10)

Topic #0:
braf akt1 ptpn11 jak1 erbb3 tp53 hras ccnd3 aurka abl1
Topic #1:
brca1 mlh1 card11 akt3 tet2 myc pim1 smad4 myd88 rhoa
Topic #2:
egfr flt3 mtor notch1 gnas tert rit1 vhl fgf3 med12
Topic #3:
kit kras fgfr3 smo ccnd1 fgfr1 araf tsc1 stat3 idh2
Topic #4:
pdgfra ret fgfr2 ntrk1 rac1 ros1 runx1 hnf1a jun map2k1
Topic #5:
erbb2 jak2 map2k1 ctnnb1 nfe2l2 brca2 whsc1l1 mef2b fgf4 cdk8
Topic #6:
pik3ca met raf1 ddr2 esr1 akt2 ezh2 pdgfrb rb1 erg
Topic #7:
brca2 erbb4 ar map2k2 pten ntrk3 sos1 idh1 pik3r2 smad2
Topic #8:
alk nras sf3b1 kdr ewsr1 casp8 rab35 pik3cb pik3cd dusp4



In [109]:
tfCA_vectorizer = CountVectorizer(stop_words='english')
tf_CA = tfCA_vectorizer.fit_transform(train_A['Variation'])
tfCB_vectorizer = CountVectorizer(stop_words='english')
tf_CB = tfCB_vectorizer.fit_transform(train_B['Variation'])
tfCC_vectorizer = CountVectorizer(stop_words='english')
tf_CC = tfCC_vectorizer.fit_transform(train_C['Variation'])

In [110]:
lda.fit(tf_CA)
lda.fit(tf_CB)
lda.fit(tf_CC)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=9, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [111]:
tfCA_feature_names = tfCA_vectorizer.get_feature_names()
tfCA_feature_names

['126',
 '13',
 '1_2009trunc',
 '385_418del',
 '51',
 '596_619splice',
 '6a',
 'a1022e',
 'a111p',
 'a120s',
 'a121e',
 'a121p',
 'a122',
 'a126d',
 'a126g',
 'a126s',
 'a126v',
 'a134d',
 'a149p',
 'a159t',
 'a161s',
 'a161t',
 'a1685s',
 'a1708e',
 'a1708v',
 'a1752p',
 'a1752v',
 'a1789t',
 'a1843p',
 'a1843t',
 'a197t',
 'a205t',
 'a209t',
 'a211d',
 'a23e',
 'a290t',
 'a298t',
 'a347t',
 'a349p',
 'a34d',
 'a36p',
 'a39p',
 'a40e',
 'a4419s',
 'a532h',
 'a57v',
 'a598t',
 'a60v',
 'a614d',
 'a617t',
 'a634v',
 'a636p',
 'a648t',
 'a707t',
 'a717g',
 'a889p',
 'a95d',
 'binding',
 'c105f',
 'c124n',
 'c124r',
 'c124s',
 'c134w',
 'c135r',
 'c135s',
 'c136r',
 'c136y',
 'c1385',
 'c141y',
 'c1697r',
 'c176f',
 'c229r',
 'c238f',
 'c238s',
 'c242f',
 'c242s',
 'c24r',
 'c275s',
 'c277q',
 'c277r',
 'c277w',
 'c278f',
 'c27a',
 'c324y',
 'c381a',
 'c384r',
 'c396r',
 'c39y',
 'c41y',
 'c44f',
 'c44y',
 'c47g',
 'c569y',
 'c61g',
 'c64g',
 'c696y',
 'c706f',
 'c71y',
 'c91a',
 'c91s',


In [112]:
tfCB_feature_names = tfCB_vectorizer.get_feature_names()
tfCB_feature_names

['17',
 '2010_2471trunc',
 '533_534del',
 '534_536del',
 'a1020v',
 'a1066v',
 'a1234t',
 'a1374v',
 'a146v',
 'a1669s',
 'a1789s',
 'a1823t',
 'a1830t',
 'a2034v',
 'a2425t',
 'a263v',
 'a2643g',
 'a272v',
 'a389t',
 'a500t',
 'a504_y505ins',
 'a530t',
 'a530v',
 'a59t',
 'a627t',
 'a633t',
 'a727v',
 'a767_v769del',
 'a77t',
 'a829p',
 'a859_l883delinsv',
 'a8s',
 'a919v',
 'acpp',
 'alk',
 'amplification',
 'atf1',
 'atf7ip',
 'atg7',
 'bcan',
 'bin2',
 'braf',
 'c1767s',
 'c1787s',
 'c2060g',
 'c360r',
 'c378r',
 'c482r',
 'c582f',
 'c634w',
 'c712r',
 'c77f',
 'c809g',
 'cad',
 'ccdc6',
 'cep85l',
 'chtop',
 'creb1',
 'cul1',
 'd1010h',
 'd1010n',
 'd1010y',
 'd101y',
 'd1029y',
 'd1091n',
 'd119n',
 'd1203n',
 'd1349h',
 'd1692n',
 'd1733g',
 'd1778g',
 'd1778n',
 'd1778y',
 'd1853n',
 'd2033n',
 'd245v',
 'd2665g',
 'd277h',
 'd287h',
 'd289_d292del',
 'd32y',
 'd384n',
 'd387v',
 'd473g',
 'd473h',
 'd579del',
 'd594g',
 'd594y',
 'd631a',
 'd631g',
 'd646y',
 'd65n',
 'd761y',

In [113]:
tfCC_feature_names = tfCC_vectorizer.get_feature_names()
tfCC_feature_names

['11',
 '19',
 '20',
 '256_286trunc',
 '422_605trunc',
 '550_592del',
 '560_561inser',
 '963_d1010splice',
 '981_1028splice',
 'a1065t',
 'a1099t',
 'a1131t',
 'a113_splice',
 'a1170v',
 'a11_g12insga',
 'a1200v',
 'a121v',
 'a1459p',
 'a146t',
 'a146v',
 'a148t',
 'a1519t',
 'a151t',
 'a1701p',
 'a171v',
 'a18d',
 'a19v',
 'a232v',
 'a2351g',
 'a246p',
 'a2717s',
 'a2770t',
 'a339v',
 'a391e',
 'a41p',
 'a41t',
 'a459v',
 'a502_y503dup',
 'a546d',
 'a598v',
 'a59g',
 'a633v',
 'a634d',
 'a677g',
 'a723d',
 'a728v',
 'a72s',
 'a72v',
 'a750_e758del',
 'a750_e758delinsp',
 'a750p',
 'a75p',
 'a763_y764insfqea',
 'a767_v769dup',
 'a77p',
 'a77s',
 'a864t',
 'a883f',
 'a883t',
 'abl1',
 'agk',
 'ahcyl1',
 'akap9',
 'akt3',
 'alk',
 'amplification',
 'ar',
 'arv567es',
 'atf7ip',
 'baiap2l1',
 'bcl2',
 'bcr',
 'bicc1',
 'braf',
 'brd4',
 'btbd1',
 'c1156f',
 'c1156y',
 'c121s',
 'c125s',
 'c1265s',
 'c135y',
 'c1365y',
 'c1483f',
 'c1483r',
 'c1483w',
 'c1483y',
 'c157y',
 'c18y',
 'c228t'

In [114]:
print_top_words(lda, tfCC_feature_names, 10)

Topic #0:
c618r i638f c620r a883f r886w g13c w557r a11_g12insga t417_d419delinsrg i1616n
Topic #1:
a146t i448v l239r s259a t286a y32s y418_d419del v14i a146v l1678p
Topic #2:
q61l e17k q61k a883t s259f y791f splice variant g325a r174
Topic #3:
amplification overexpression g13d t654i g253c l790f c609y v654a v555_v559del l19f
Topic #4:
g12v f28l s427g r661w g12s l63v a151t s222d p551_e554del p29l
Topic #5:
exon q61r deletion g12d 20 19 insertion q209l t654m r342w
Topic #6:
fusion ntrk1 braf ntrk3 ewsr1 fgfr1 fgfr2 etv6 jak2 ret
Topic #7:
mutations pdgfra g42r r276w g60e c634r c248t r342q g505s truncating
Topic #8:
q61h v804m s891a w563l s257w c630r c634s c634y g774v e542k



In [115]:
XA_final = sp.hstack((tf_AA,tf_BA,tf_CA))

In [116]:
XB_final = sp.hstack((tf_AB,tf_BB,tf_CB))

In [117]:
XC_final = sp.hstack((tf_AC,tf_BC,tf_CC))

In [118]:
XA_final.shape

(1254, 88075)

In [119]:
Y_A.shape

(1254,)

In [120]:
XA_train, XA_test, YA_train, YA_test = train_test_split(XA_final, Y_A, random_state=1)

In [121]:
# defining parameter range 
param_grid = {'C': [0.1, 1], #10, 100, 1000],  
              'gamma': [1, 0.1, 0.01], #0.001, 0.0001], 
              'kernel': ['linear']}  
  
gridA = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,) 
  
# fitting the model for grid search 
gridA.fit(XA_train, YA_train) 

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.678, total=  38.2s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   38.2s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.732, total=  46.2s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.677, total=  34.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.678, total=  39.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.732, total=  48.1s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.677, total=  35.5s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.678, total=  38.9s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.732, total=  47.4s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.677, total=  36.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 13.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [122]:
yA_pred=gridA.predict(XA_test)

In [123]:
print(classification_report(YA_test, yA_pred))

              precision    recall  f1-score   support

           1       0.67      0.75      0.71       139
           4       0.78      0.71      0.74       175

    accuracy                           0.73       314
   macro avg       0.73      0.73      0.72       314
weighted avg       0.73      0.73      0.73       314



In [124]:
XB_train, XB_test, YB_train, YB_test = train_test_split(XB_final, Y_B, random_state=1)

In [125]:
# defining parameter range 
param_grid = {'C': [0.1, 1], #10, 100, 1000],  
              'gamma': [1, 0.1, 0.01], #0.001, 0.0001], 
              'kernel': ['linear']}  
  
gridB = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,) 
  
# fitting the model for grid search 
gridB.fit(XB_train, YB_train) 

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.839, total=   4.5s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.838, total=   5.4s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    9.9s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.879, total=   4.7s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.839, total=   4.6s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.838, total=   6.0s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.879, total=   5.1s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.839, total=   4.6s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.838, total=   6.1s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.879, total=   4.7s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  1.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [126]:
yB_pred=gridB.predict(XB_test)

In [127]:
print(classification_report(YB_test, yB_pred))

              precision    recall  f1-score   support

           2       0.91      0.91      0.91       112
           5       0.84      0.84      0.84        62

    accuracy                           0.89       174
   macro avg       0.87      0.87      0.87       174
weighted avg       0.89      0.89      0.89       174



In [128]:
XC_train, XC_test, YC_train, YC_test = train_test_split(XC_final, Y_C, random_state=1)

In [129]:
param_grid = {'C': [0.1, 1], #10, 100, 1000],  
              'gamma': [1, 0.1, 0.01], #0.001, 0.0001], 
              'kernel': ['linear']}  
  
gridC = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,) 
  
# fitting the model for grid search 
gridC.fit(XC_train, YC_train) 

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.853, total=  27.2s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.2s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.835, total=  24.5s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   51.7s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.861, total=  25.8s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.853, total=  27.0s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.835, total=  25.5s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.861, total=  25.6s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.853, total=  27.2s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.835, total=  25.2s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .... C=0.1, gamma=0.01, kernel=linear, score=0.861, total=  25.6s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  7.6min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [130]:
yC_pred=gridC.predict(XC_test)

In [131]:
print(classification_report(YC_test, yC_pred))

              precision    recall  f1-score   support

           3       0.37      0.33      0.35        21
           6       0.82      0.88      0.85        69
           7       0.92      0.91      0.92       242
           9       0.86      0.86      0.86         7

    accuracy                           0.87       339
   macro avg       0.74      0.75      0.74       339
weighted avg       0.87      0.87      0.87       339



In [132]:
def evaluate(X, y, clf=None):
    probas = cross_val_predict(clf, X, y, cv=StratifiedKFold(n_splits=5, random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2)
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))

In [133]:
clf = XGBClassifier()
clf.fit(XA_train,YA_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [134]:
yA_pred=clf.predict(XA_test)

In [135]:
print(classification_report(YA_test, yA_pred))

              precision    recall  f1-score   support

           1       0.77      0.76      0.76       139
           4       0.81      0.82      0.82       175

    accuracy                           0.79       314
   macro avg       0.79      0.79      0.79       314
weighted avg       0.79      0.79      0.79       314



In [136]:
clf.fit(XB_train,YB_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [137]:
yB_pred=clf.predict(XB_test)

In [138]:
print(classification_report(YB_test, yB_pred))

              precision    recall  f1-score   support

           2       0.87      0.96      0.92       112
           5       0.92      0.74      0.82        62

    accuracy                           0.89       174
   macro avg       0.90      0.85      0.87       174
weighted avg       0.89      0.89      0.88       174



In [139]:
#Creating a stacked model

In [140]:
seed = 1075
np.random.seed(seed)
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()
clf_array = [rf, et, knn, svc, rg]
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, XA_train,YA_train, cv=10, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, 
       max_samples=0.4, max_features=10, random_state=seed)
    bagging_scores = cross_val_score(bagging_clf, XA_train,YA_train, cv=10, 
       n_jobs=-1)
    
    print (clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std())
    print (clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std())

RandomForestClassifier 0.7543055304699635 0.03294816205505427
RandomForestClassifier 0.5819388553745379 0.03291732341068577
ExtraTreesClassifier 0.7437344218473432 0.0462512674216608
ExtraTreesClassifier 0.5840667557706896 0.02906089793400136
KNeighborsClassifier 0.6851777866079063 0.032164919111819144
KNeighborsClassifier 0.5255444376211633 0.06139629328689964
SVC 0.7468925574058689 0.05064906629188679
SVC 0.5468095943358741 0.0246993668417568
RidgeClassifier 0.7202286601885635 0.03995179885459258
RidgeClassifier 0.5531925730592782 0.022188838884922884


In [141]:
seed = 1075
np.random.seed(seed)
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()
clf_array = [rf, et, knn, svc, rg]
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, XB_train,YB_train, cv=10, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, 
       max_samples=0.4, max_features=10, random_state=seed)
    bagging_scores = cross_val_score(bagging_clf, XB_train,YB_train, cv=10, 
       n_jobs=-1)
    
    print (clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std())
    print (clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std())

RandomForestClassifier 0.8865384615384617 0.03592604171590269
RandomForestClassifier 0.6615384615384616 0.009421114395319936
ExtraTreesClassifier 0.8807692307692309 0.04452244962611624
ExtraTreesClassifier 0.6692307692307693 0.014390989949130541
KNeighborsClassifier 0.8365384615384617 0.05248978486136992
KNeighborsClassifier 0.6557692307692309 0.005769230769230782
SVC 0.875 0.04884009653538574
SVC 0.653846153846154 1.1102230246251565e-16
RidgeClassifier 0.8673076923076923 0.05192307692307693
RidgeClassifier 0.653846153846154 1.1102230246251565e-16


In [142]:
seed = 1075
np.random.seed(seed)
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()
clf_array = [rf, et, knn, svc, rg]
for clf in clf_array:
    vanilla_scores = cross_val_score(clf, XC_train,YC_train, cv=10, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, 
       max_samples=0.4, max_features=10, random_state=seed)
    bagging_scores = cross_val_score(bagging_clf, XC_train,YC_train, cv=10, 
       n_jobs=-1)
    
    print (clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std())
    print (clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std())

RandomForestClassifier 0.871989784243987 0.03966336662165517
RandomForestClassifier 0.7005361877135743 0.005409310997304704
ExtraTreesClassifier 0.8680280405539126 0.03018114875115143
ExtraTreesClassifier 0.7005361877135743 0.005409310997304704
KNeighborsClassifier 0.8403433021771622 0.04460969534366396
KNeighborsClassifier 0.7005361877135743 0.005409310997304704
SVC 0.8641074313028104 0.035633920792259716
SVC 0.7005361877135743 0.005409310997304704
RidgeClassifier 0.8720573495956107 0.02872127620374133
RidgeClassifier 0.7005361877135743 0.005409310997304704


In [143]:
ada_boost = AdaBoostClassifier()
grad_boost = GradientBoostingClassifier()
xgb_boost = XGBClassifier()
boost_array = [ada_boost, grad_boost, xgb_boost]
eclf = EnsembleVoteClassifier(clfs=[ada_boost, grad_boost, xgb_boost], voting='hard')
labels = ['Ada Boost', 'Grad Boost', 'XG Boost', 'Ensemble']
for clf, label in zip([ada_boost, grad_boost, xgb_boost, eclf], labels):
    scores = cross_val_score(clf, XA_train,YA_train, cv=10, scoring='accuracy')
    print((scores.mean(), scores.std(), label))

(0.753116834639791, 0.0550764420333092, 'Ada Boost')
(0.7808445616443305, 0.05240301712670176, 'Grad Boost')
(0.7819541475514455, 0.04900361043389388, 'XG Boost')
(0.7702179436236438, 0.052974739158935205, 'Ensemble')


In [144]:
eclf = EnsembleVoteClassifier(clfs=[rf, et, knn, svc, ada_boost, grad_boost, xgb_boost], voting='hard')

In [145]:
eclf.fit(XA_train,YA_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


EnsembleVoteClassifier(clfs=[RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None,
                                                    oob_